# Estimating Baseline Performance

### What is a baseline model? 

Producing a baseline model is crucial for evaluating your model's performance on any machine learning problem. A baseline model is a basic solution to a machine learning problem that serves as a point of reference for comparing other models to. The baseline model's performance gives us an indication of how much better our models can perform relative to a naive approach. 

Let's say we are building a sentence similarity model where our training set contains pairs of sentences and we want to predict how similiar these sentences are on a scale from 1-5. We could spend months producing a complex machine learning solution to this problem and ultimately get a mean squared error (MSE) of 0.3. But is this result good or bad? There is no way of knowing without comparing it with some baseline performance. For our baseline model, we could predict the mean sentence similarity of sentence pairs in our training set (called the _zero rule_) and get a MSE of 0.35. So our model is worse than the baseline which indicates that we may want to consider using different features, models, evaluation metrics, etc. It is crucial that the choice of baseline model be tailored to a data science problem based on buisness goals and the specific modeling task.

### What are good baselines for sentence similarity?

For sentence similarity problems, we have two sub-tasks: 1) First, we need to produce a vector representation of each sentence in the sentence pair, known as an **embedding**. 2) Second, we need to compute the similarity between these two sentence embeddings.

For producing representations of sentences, there are some common baseline approaches: 
1. Create word embeddings for each word in a sentence
    1. word2vec word embeddings
    2. GLoVe word embeddings
    3. fastText word embeddings
    
2. Create sentence embeddings
    1. doc2vec document embeddings
    2. TF-IDF embeddings 

Then we have to compare our embeddings to calculate sentence similarity:
1. Word Embedding comparison
    1. Cosine Similarity (first requires averaging the word embeddings of all words in each sentence)
    2. Word Mover's Distance

2. Sentence Embedding comparison
    1. Cosine Similarity  
    
The different embedding models and similarity metrics are introduced below.

# Table of Contents
* [Data Loading and Preprocessing](#Data-Loading-and-Preprocessing)
    * [Load STS Benchmark Dataset](#Load-STS-Benchmark-Dataset)
    - [Preprocess / Tokenize](#Data-Preprocessing-/-Tokenization)
    - [Document Frequency Calculation](#Document-Frequency-Calculation)
* [Baseline Models](#Baseline-Models)
    - [Baseline #1: word2vec and cosine similarity](#Baseline-#1:-word2vec-embeddings-+-cosine-similarity-(word2vec-+-cosine))
    - [Baseline #2: word2vec and Word Mover's Distance](#Baseline-#2:-word2vec-embeddings-+-Word-Mover's-Distance-(word2vec-+-WMD))
    - [Baseline #3: GloVe and cosine similarity](#Baseline-#3:-GloVe-embeddings-+-cosine-similarity-(GloVe-+-cosine))
    * [Baseline #4: GloVe and Word Mover's Distance](#Baseline-#4:-GloVe-embeddings-+-Word-Mover's-Distance-(GloVe-+-WMD))
    - [Baseline #5: fastText and cosine similarity](#Baseline-#5:-fastText-embeddings-+-cosine-similarity-(fastText-+-cosine))
    - [Baseline #6: fastText and Word Mover's Distance](#Baseline-#6:-fastText-embeddings-+-Word-Mover's-Distance-(fastText-+-WMD))

    * [Baseline #7: TF-IDF and cosine similarity](#Baseline-#7:-TF-IDF-embeddings-+-cosine-similarity-(TF-IDF-+-cosine))
    * [Baseline #8: Doc2vec and cosine similarity](#Baseline-#8:-Doc2vec-embeddings-+-cosine-similarity-(Doc2vec-+-cosine))

* [Comparison of Baseline Models](#Comparison-of-Baseline-Models)

In [44]:
#Import Packages
import sys
import os
from collections import Counter
import math
import numpy as np
import scipy
from scipy.spatial import distance
import gensim
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

# Data Loading and Preprocessing

### Load STS Benchmark Dataset

Here we utilize the [STS Benchmark dataset](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark#STS_benchmark_dataset_and_companion_dataset) which contains a selection of English datasets that were used in Semantic Textual Similarity (STS) tasks 2012-2017. The datasets include text from image captions, news headlines, and user forums. The dataset contains 8,628 sentence pairs with a human-labeled integer representing the sentences' similarity (ranging from 0, for no meaning overlap, to 5, meaning equivalence).

In [45]:
sys.path.append("../../../")  ## set the environment path
BASE_DATA_PATH = "../../../data"

from utils_nlp.dataset.preprocess import to_lowercase, to_spacy_tokens
from utils_nlp.dataset import stsbenchmark

In [46]:
# Produce a pandas dataframe for the training and test sets
stsTrain = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")
stsTest = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="test")

In [47]:
print(f"Training set has {len(stsTrain)} sentences")
print(f"Testing set has {len(stsTest)} sentences")

Training set has 5749 sentences
Testing set has 1379 sentences


In [48]:
stsTest.head(10)

,score,sentence1,sentence2
0,2.500,A girl is styling her hair.,A girl is brushing her hair.
1,3.600,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
2,5.000,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
3,4.200,A man is cutting up a cucumber.,A man is slicing a cucumber.
4,1.500,A man is playing a harp.,A man is playing a keyboard.
5,1.800,A woman is cutting onions.,A woman is cutting tofu.
6,3.500,A man is riding an electric bicycle.,A man is riding a bicycle.
7,2.200,A man is playing the drums.,A man is playing the guitar.
8,2.200,A man is playing guitar.,A lady is playing the guitar.
9,1.714,A man is playing a guitar.,A man is playing a trumpet.


### Data Preprocessing / Tokenization

Our baseline models will expect that each sentence is represented by a list of **tokens**. Tokens are linguistic units like words, punctuation marks, numbers, etc. We'll use our util functions which utilize the spaCy package, a popular package for performing tokenization.

It's also common to remove high-frequency words which do not help distinguish one sentence from another, so called **stop words**. For example, "the", "and", "a", etc. are typical stop words although each tokenization package may differ in the words they consider to be stop words. We'll tokenize our corpus with and without stop words so that we can compare our methods with and without stop words.

In [49]:
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)

In [50]:
# train preprocessing
df_low = to_lowercase(stsTrain)  # covert all text to lowercase
sts_tokenize = to_spacy_tokens(df_low)  # tokenize normally
sts_train_stop = rm_spacy_stopwords(sts_tokenize)  # tokenize with removal of stopwords

Now each row in our dataframe contains:  
- The similarity score of the sentence pair
- The 2 original sentences from our datasets  
- A column for each sentence's tokenization with stop words  
- A column for each sentence's tokenization without stop words

In [51]:
sts_train_stop.head(5)

,score,sentence1,sentence2,sentence1_tokens,sentence2_tokens,sentence1_tokens_stop,sentence2_tokens_stop
0,5.00,a plane is taking off.,an air plane is taking off.,"[a, plane, is, taking, off, .]","[an, air, plane, is, taking, off, .]","[plane, taking, .]","[air, plane, taking, .]"
1,3.80,a man is playing a large flute.,a man is playing a flute.,"[a, man, is, playing, a, large, flute, .]","[a, man, is, playing, a, flute, .]","[man, playing, large, flute, .]","[man, playing, flute, .]"
2,3.80,a man is spreading shreded cheese on a pizza.,a man is spreading shredded cheese on an uncoo...,"[a, man, is, spreading, shreded, cheese, on, a...","[a, man, is, spreading, shredded, cheese, on, ...","[man, spreading, shreded, cheese, pizza, .]","[man, spreading, shredded, cheese, uncooked, p..."
3,2.60,three men are playing chess.,two men are playing chess.,"[three, men, are, playing, chess, .]","[two, men, are, playing, chess, .]","[men, playing, chess, .]","[men, playing, chess, .]"
4,4.25,a man is playing the cello.,a man seated is playing the cello.,"[a, man, is, playing, the, cello, .]","[a, man, seated, is, playing, the, cello, .]","[man, playing, cello, .]","[man, seated, playing, cello, .]"


In [52]:
# Repeat process to perform preprocessing for test set
df_low = to_lowercase(stsTest)
sts_tokenize = to_spacy_tokens(df_low)
sts_test_stop = rm_spacy_stopwords(sts_tokenize)

### Document Frequency Calculation

Many of the baseline models we'll explore will require calculation of how frequently a word appears in the sentences of our corpus. In this preprocessing step, we iterate through the sentences in our training set, counting the number of sentences that contain each word. There are other ways to produce this calculation, including pulling larger datasets from the web (like Wikipedia data) and calculating the frequency on that data. Note that "document" refers to some larger chunk of multiple tokens/words. In our case, our documents will actually be individual sentences. 

In [53]:
def get_document_frequency(df):
    """Iterate through all sentences in dataframe df and create dictionary mapping tokens to the number of sentences
    they appear in in our corpus
    
    Args:
        df (pandas dataframe): dataframe of sentence pairs with their similarity scores
        
    Returns:
        document_frequency_dict (dictionary): mapping from tokens to number of sentences they appear in
        n (int): number of sentences in the corpus
    """
    document_frequency_dict = {}
    sentences = df["sentence1_tokens"].append(df["sentence2_tokens"])

    for s in sentences:
        for token in set(s):
            if token in document_frequency_dict:
                document_frequency_dict[token] += 1
            else:
                document_frequency_dict[token] = 1
    n = len(sentences)
    return document_frequency_dict, n

In [54]:
# note that we need to calculate these values on our training set so that we don't "peek at" our test set until test time
document_frequencies, num_documents = get_document_frequency(sts_train_stop)

In [55]:
num_documents

11498

# Baseline Models

As we consider each of the baseline models, we'll save all model predictions in a dictionary and will evaluate the results at the end of this notebook.

In [56]:
baselines = {}

## Baseline #1: word2vec embeddings + cosine similarity (word2vec + cosine)

This baseline first constructs word embeddings using word2vec. Once we have a word embedding (vector) for each word in the sentence, we calculate an embedding for the full sentence by taking the (weighted) average of all the word embeddings. The weights will be calculated using TF-IDF. Lastly, in order to compare the two sentence embeddings we use the cosine similarity metric. 

### What is Word2Vec?
Word2vec is a predictive model for learning word embeddings from text. Word embeddings are learned such that words that share common contexts in the corpus will be close together in the vector space. There are two different model architectures that can be used to produce word2vec embeddings: continuous bag-of-words (CBOW) or continuous skip-gram. The former uses a window of surrounding words (the "context") to predict the current word and the latter uses the current word to predict the surrounding context words. See this [tutorial](https://www.guru99.com/word-embedding-word2vec.html#3) on word2vec for more detailed background on the model.

For our purposes, we use pretrained word2vec word embeddings. These embeddings were trained on a Google News corpus and provide 300-dimensional embeddings (vectors) for 3 million English words. See this link for the original source (https://code.google.com/archive/p/word2vec/) and see the code below to load these word embeddings.

In [4]:
from utils_nlp.pretrained_embeddings import word2vec

word2vec_model = word2vec.load_pretrained_vectors(dir_path=BASE_DATA_PATH)

1.65GB [01:00, 27.3MB/s]                                                                                               


<class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>


### What is TF-IDF?

TF-IDF or term frequency-inverse document frequency is a weighting scheme intended to measure how important a word is to the document (or sentence in our case) within the broader corpus (our dataset). The weight "increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus" ([tutorial link](http://www.tfidf.com/)). When we're averaging together many different word vectors to get a sentence embedding, it makes sense to give stronger weight to words that are more distinct relative to the corpus and that have a high frequency in the sentence. The TF-IDF weights capture this intution, with the weight increasing as term frequency increases and/or as the inverse document frequency increases.

For a term $t$ in sentence $s$ in corpus $c$, then the TF-IDF weight is 
$$w_{t,s} = TF_{t,s} * \log{\frac{N}{df_t}}$$
where:  
$TF_{t,s}$ = the number of times term $t$ appears in sentence $s$  
$df_t$ = the number of sentences containing term $t$  
$N$ = the size of the corpus.  

In these baselines, we calculate the TF-IDF weighted average of all the word embeddings. The code below implements this weighted average given a list of tokens and an embedding model.

In [17]:
def average_sentence_embedding(tokens, embedding_model):
    """Calculate TF-IDF weighted average embedding for a sentence
    
    Args:
        tokens (list): list of tokens in a sentence
        embedding_model (gensim model): model to use for word embedding (word2vec, glove, fastText, etc.)
    
    Returns:
        list: vector representing the sentence
    """
    # throw away tokens that are not in embeddings model
    tokens = [i for i in tokens if i in embedding_model]

    if len(tokens) == 0:
        return []

    # We will weight by TF-IDF:
    # For the TF part: # of times term appears / total terms in sentence
    count = Counter(tokens)
    token_list = list(count)
    term_frequency = [count[i] / len(tokens) for i in token_list]

    # Now for the IDF part: LOG(# documents / # documents with term in it)
    inv_doc_frequency = [
        math.log(num_documents / (document_frequencies.get(i, 0) + 1)) for i in count
    ]

    # Put the TF-IDF together and produce the weighted average of vector embeddings
    word_embeddings = [embedding_model[token] for token in token_list]
    weights = [term_frequency[i] * inv_doc_frequency[i] for i in range(len(token_list))]
    return list(np.average(word_embeddings, weights=weights, axis=0))

### What is Cosine Similarity?

Cosine similarity is a common similarity metric between vectors. Intuitively it measures the cosine of the angle between any two vectors. With vectors $a$ and $b$, the cosine similarity is: cosine similarity($a$,$b$) = $\frac{\vec{a} \cdot \vec{b} }{||\vec{a}|| ||\vec{b}||}$


In [18]:
def calculate_cosine_similarity(embedding1, embedding2):
    """Calculate cosine similarity between two embedding vectors
    
    Args:
        embedding1 (list): embedding for the first sentence
        embedding2 (list): embedding for the second sentence
    
    Returns:
        list: cosine similarity value between the two embeddings
    """
    # distance.cosine calculates cosine DISTANCE, so take 1 - distance to get cosine similarity
    cosine_similarity = 1 - distance.cosine(embedding1, embedding2)
    return cosine_similarity

### Get Sentence Similarity Predictions for Test Set

Now, we calculate predictions for each of sentence pairs found in the test set.

In [19]:
def average_word_embedding_cosine_similarity(df, embedding_model, rm_stopwords=False):
    """Calculate the cosine similarity between TF-IDF weighted averaged embeddings
    
    Args:
        df (pandas dataframe): dataframe as provided by the nlp_utils
        embedding_model (gensim model): word embedding model
        rm_stopwords (bool): whether to use stop words (True) or remove them (False)
    
    Returns:
        list: predicted values for sentence similarity of test set examples
    """
    predictions = []
    if rm_stopwords:
        tokenized_sentences = zip(
            df["sentence1_tokens_stop"], df["sentence2_tokens_stop"]
        )
    else:
        tokenized_sentences = zip(df["sentence1_tokens"], df["sentence2_tokens"])


    for (sentence1, sentence2) in tokenized_sentences:
        embedding1 = average_sentence_embedding(sentence1, embedding_model)
        embedding2 = average_sentence_embedding(sentence2, embedding_model)
        if embedding1 == [] or embedding2 == []:
            predictions.append(0)
        else:
            predictions.append(calculate_cosine_similarity(embedding1, embedding2))
    return predictions

In [20]:
# get predictions using average word2vec embeddings both with and without stop words
baselines["Word2vec Cosine"] = average_word_embedding_cosine_similarity(
    sts_test_stop, word2vec_model, rm_stopwords=True
)
baselines["Word2vec Cosine with Stop Words"] = average_word_embedding_cosine_similarity(
    sts_test_stop, word2vec_model, rm_stopwords=False
)

## Baseline #2: word2vec embeddings + Word Mover's Distance (word2vec + WMD)

This baseline first constructs word embeddings using word2vec (for an introduction to word2vec, see [Background on Word2Vec](#What-is-Word2Vec?)). Then all the word embeddings are used to calculate sentence similarity using the word mover's distance.  

### What is Word Mover's Distance (WMD)?
Word Mover's Distance (WMD) is a metric that "adapts the earth mover’s distance to the space of documents: the distance between two texts is given by the total amount of “mass” needed to move the words from one side into the other, multiplied by the distance the words need to move." We'll utilize word2vec's implementation of word mover's distance. See this [blog](http://vene.ro/blog/word-movers-distance-in-python.html) for additional information about this similarity measure.

### Get Sentence Similarity Predictions for Test Set

Now, we calculate predictions for each of sentence pairs found in the test set.

In [21]:
def word_embedding_WMD(df, embedding_model, rm_stopwords=False):
    """Calculate Word Mover's Distance between two sentences using embeddings
    
    Args:
        df (pandas dataframe): dataframe as provided by the nlp_utils
        embedding_model (gensim model): word embedding model
        rm_stopwords (bool): whether to use stop words (True) or remove them (False)
    
    Returns:
        list: predicted values for sentence similarity of test set examples
    """
    predictions = []
    if rm_stopwords:
        tokenized_sentences = zip(
            df["sentence1_tokens_stop"], df["sentence2_tokens_stop"]
        )
    else:
        tokenized_sentences = zip(df["sentence1_tokens"], df["sentence2_tokens"])

    for (sentence1, sentence2) in tokenized_sentences:
        # throw away tokens that are not in embeddings model
        tokens1 = [i for i in sentence1 if i in embedding_model]
        tokens2 = [i for i in sentence2 if i in embedding_model]
        if tokens1 == [] or tokens2 == []:
            predictions.append(0)
        else:
            # wmdistance takes the raw tokens and performs the word2vec embedding itself
            predictions.append(-embedding_model.wmdistance(tokens1, tokens2))
    return predictions

In [23]:
# get predictions using word2vec embeddings and WMD both with and without stop words
baselines["Word2vec WMD"] = word_embedding_WMD(sts_test_stop, word2vec_model, rm_stopwords=True)
baselines["Word2vec WMD with Stop Words"] = word_embedding_WMD(
    sts_test_stop, word2vec_model, rm_stopwords=False
)

## Baseline #3: GloVe embeddings + cosine similarity (GloVe + cosine)

This baseline first constructs word embeddings using GloVE. Once we have a word embedding (vector) for each word in the sentence, we calculate an embedding for the full sentence by taking the (weighted) average of all the word embeddings. The weights will be calculated using TF-IDF. Lastly, in order to compare the two sentence embeddings we use the cosine similarity metric (for an introduction to the cosine similarity metric, see [Background on Cosine Similarity](#What-is-Cosine-Similarity?)). 

### What is GloVe?
GloVe is an unsupervised algorithm for obtaining word embeddings. Training occurs on word-word co-occurance statistics with the objective of learning word embeddings such that the dot product of two word's embeddings is equal to the word's probability of co-occurance. See this [tutorial](https://nlp.stanford.edu/projects/glove/) on GloVe for more detailed background on the model. For our purposes, we use pretrained GloVe word embeddings (glove.840B.300d.zip which can be downloaded from above link). These embeddings were trained on Common Crawl data and provide 300-dimensional embeddings (vectors) for 2.2 million English words. Below is the code to load in the GloVe embeddings.

In [26]:
from gensim.scripts.glove2word2vec import glove2word2vec

tmp_file = "glove.840B.300d.w2v.txt"

In [27]:
# we need to download the GLoVe file and convert it to word2vec format, this takes a bit of time
PATH_TO_GLOVE = os.path.expanduser("glove.840B.300d.txt")
glove2word2vec(PATH_TO_GLOVE, tmp_file)

FileNotFoundError: [Errno 2] No such file or directory: 'glove.840B.300d.txt'

In [24]:
# Once the above cell has been run once, just need to use this command to load the embeddings
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)

In [25]:
# get predictions using glove embeddings and cosine similarity both with and without stop words
baselines["GLoVe Cosine"] = average_word_embedding_cosine_similarity(
    sts_test_stop, glove, rm_stopwords=True
)
baselines["GLoVe Cosine with Stop Words"] = average_word_embedding_cosine_similarity(
    sts_test_stop, glove, rm_stopwords=False
)

## Baseline #4: GloVe embeddings + Word Mover's Distance (GloVe + WMD)

This baseline first constructs word embeddings using GloVe (for an introduction on GloVe, see [Background on GloVe](#What-is-GloVe?)). Then all the word embeddings are used to calculate sentence similarity using the word mover's distance (for an introduction to WMD, see [Background on Word Mover's Distance](#What-is-Word-Mover's-Distance-(WMD)?)).  

In [26]:
# get predictions using glove embeddings and WMD both with and without stop words
baselines["GLoVe WMD"] = word_embedding_WMD(sts_test_stop, glove, rm_stopwords=True)
baselines["GLoVe WMD with Stop Words"] = word_embedding_WMD(
    sts_test_stop, glove, rm_stopwords=False
)

## Baseline #5: fastText embeddings + cosine similarity (fastText + cosine)

This baseline first constructs word embeddings using fastText. Once we have a word embedding (vector) for each word in the sentence, we calculate an embedding for the full sentence by taking the (weighted) average of all the word embeddings. The weights will be calculated using TF-IDF. Lastly, in order to compare the two sentence embeddings we use the cosine similarity metric (for an introduction to the cosine similarity metric, see [Background on Cosine Similarity](#What-is-Cosine-Similarity?)). 

### What is fastText?

fastText is an unsupervised algorithm created by Facebook Research for efficiently learning word embeddings. fastText is significantly different than word2vec or GloVe in that these two algorithms we saw earlier treat each word as the smallest possible unit to find an embedding for. Conversely, fastText assumes that words are formed by an n-gram of characters (i.e. 2-grams of the word "language" would be {la, an, ng, gu, ua, ag, ge}). The embedding for a word is then composed of the sum of these character n-grams. This has advantages when finding word embeddings for rare words and words not present in the dictionary, as these words can still be broken down into character n-grams. Typically, for smaller datasets, fastText performs better than word2vec or GloVe. See this [tutorial](https://fasttext.cc/docs/en/unsupervised-tutorial.html) on fastText for more detail. We will use the pretrained word embeddings for the English language (wiki.en.bin; these embeddings as well as embeddings for 156 other languages can be found at https://fasttext.cc/docs/en/english-vectors.html). These are 300-dimensional embeddings (vectors) trained on Wikipedia data.

In [27]:
fastText_model = gensim.models.fasttext.load_facebook_model("wiki.en\wiki.en.bin")
fastText = fastText_model.wv  # get word vectors

In [28]:
# get predictions using fastText embeddings and cosine similarity both with and without stop words
baselines["fastText Cosine"] = average_word_embedding_cosine_similarity(
    sts_test_stop, fastText, rm_stopwords=True
)
baselines["fastText Cosine with Stop Words"] = average_word_embedding_cosine_similarity(
    sts_test_stop, fastText, rm_stopwords=False
)

## Baseline #6: fastText embeddings + Word Mover's Distance (fastText + WMD)

In [29]:
# get predictions using fastText embeddings and WMD both with and without stop words
baselines["fastText WMD"] = word_embedding_WMD(sts_test_stop, fastText, rm_stopwords=True)
baselines["fastText WMD with Stop Words"] = word_embedding_WMD(
    sts_test_stop, fastText, rm_stopwords=False
)

## Baseline #7: TF-IDF embeddings + cosine similarity (TF-IDF + cosine)

This baseline first constructs a document embedding based on bag of words with TF-IDF weighting (for an introduction to TF-IDF, see [Background on TF-IDF](#What-is-TF-IDF?). Then we apply cosine similarity between the two embeddings in the sentence pair (for an introduction to the cosine similarity metric, see [Background on Cosine Similarity](#What-is-Cosine-Similarity?)).

### Bag of Words

The most basic approach for document embeddings is called Bag-of-Words. This method first determines the vocabulary across the entire corpus and then, for each document, creates a vector containing the number of times each vocabulary word appeared in the given document. These vectors are obviously very sparse and typical bag of words implementations ignore terms whose document frequency is less than some threshold in order to reduce sparsity. We also often ignore stop words as they add little semantic information. 

In [106]:
def tfidf_cosine_similarity(df, rm_stopwords=False):
    """Calculate cosine similarity between TF-IDF document embeddings
    
    Args:
        df (pandas dataframe): dataframe as provided by the nlp_utils
        rm_stopwords (bool): whether to remove stop words or not
    
    Returns:
        list: predicted values for sentence similarity of test set examples
    """
    stop_word_param = 'english' if rm_stopwords else None
    
    tf = TfidfVectorizer(
        input="content",
        analyzer="word",
        min_df=0,
        stop_words=stop_word_param,
        sublinear_tf=True,
    )

    all_sentences =  df[["sentence1", "sentence2"]]
    corpus = all_sentences.values.flatten().tolist()
    tfidf_matrix = np.array(tf.fit_transform(corpus).todense())
    
    df['sentence1_tfidf'] = df.apply(lambda x: tfidf_matrix[2*x.name,:], axis=1)
    df['sentence2_tfidf'] = df.apply(lambda x: tfidf_matrix[2*x.name+1,:], axis=1)
    df['predictions'] = df.apply(lambda x: calculate_cosine_similarity(x.sentence1_tfidf, x.sentence2_tfidf) if 
                                 (sum(x.sentence1_tfidf) != 0 and sum(x.sentence2_tfidf) != 0) else 0,axis=1)
    return df['predictions'].tolist()

In [76]:
import pandas as pd

In [81]:
ex = pd.DataFrame({'a':[1,2,3],'b':[4,5,6]})
ex

,a,b
0,1,4
1,2,5
2,3,6


In [100]:
ex['sent1'] = ex.apply(lambda x: 2*x.name,axis=1)
ex['sent2'] = ex.apply(lambda x: 2*x.name+1,axis=1)

ex

,a,b,index,sent1,sent2
0,1,4,0,0,1
1,2,5,1,2,3
2,3,6,2,4,5


In [107]:
baselines["TF-IDF Cosine"] = tfidf_cosine_similarity(sts_test_stop, rm_stopwords=True)
baselines["TF-IDF Cosine with Stop Words"] = tfidf_cosine_similarity(
    sts_test_stop, rm_stopwords=False
)

In [108]:
baselines["TF-IDF Cosine with Stop Words"]

[0.5154669367957403,
 0.6238376709093325,
 0.5983062335986293,
 0.7102615298532333,
 0.3351548713228747,
 0.4586765374374362,
 0.7453648441043782,
 0.4332143047353778,
 0.6274081602186135,
 0.4102968379263213,
 0.4102968379263213,
 0.5253589511249759,
 0.17651528643337078,
 0.5499522389851709,
 0.3787193514643298,
 0.4332143047353778,
 0.5423224039397149,
 0.7519050657939645,
 0.30617844695350493,
 0.12920385409392754,
 0.5115388235096275,
 0.12571551261385527,
 0.671208418075113,
 0.6147994882540094,
 0.3085863437907099,
 0.8665043113349088,
 0.3499638723495143,
 0.4485303934478104,
 0.14412070093774276,
 0.7223992863241857,
 0.42977927537233007,
 0.6687393429045153,
 0.09883942492880626,
 0.8006601194016243,
 0.3915728034097792,
 0.12959796544910263,
 0.07074855491798504,
 0.24392238095706376,
 0.5797213695434924,
 0.1455830629990571,
 0.5774415169951077,
 0.7562038442807758,
 0.7519050657939645,
 0.14623990642389495,
 0.271411211816421,
 0.10679246383512553,
 0.12436018172562091,
 0

## Baseline #8: Doc2vec embeddings + cosine similarity (Doc2vec + cosine)

This baseline constructs document embeddings using doc2vec and then applies cosine similarity to measure each sentence pair's similarity (for an introduction to the cosine similarity metric, see [Background on Cosine Similarity](#What-is-Cosine-Similarity?)).

### What is Doc2Vec?

Doc2vec is an extension of word2vec which produces embeddings of a document. Note that "document" refers to some larger chunk of multiple tokens/words. In our case, our documents will actually be individual setntences. The algorithm not only exploits the idea of context words (like in word2vec), but also incorporates the context of the document. There are again two model architectures that parallel those of word2vec: Paragraph Vectors Distributed Memory (PV-DM) and Paragraph Vectors Distributed Bag-of-Words (PV-DBOW). PV-DM randomly samples consecutive words in a paragraph and predicts a center word by utilizing the context words and the paragraph id. PV-DBOW takes a paragraph id and uses it to predict words in the context. 

See [tutorial #1](https://kanoki.org/2019/03/07/sentence-similarity-in-python-using-doc2vec/) or [tutorial #2](https://gab41.lab41.org/doc2vec-to-assess-semantic-similarity-in-source-code-667acb3e62d7) for more information and an example of using Doc2vec for sentence similarity.

In [32]:
# doc2vec requires unique ids for each sentence, so we'll iterate through making a dictionary of sentence to its id
id_dict = {}
n = 0


def assign_id(row):
    global n
    if row not in id_dict:
        id_dict[row] = n
        n += 1
    return id_dict[row]


sts_test_stop["qid1"] = sts_test_stop["sentence1"].apply(assign_id)
sts_test_stop["qid2"] = sts_test_stop["sentence2"].apply(assign_id)

In [33]:
def doc2vec_cosine(df, rm_stopwords=False):
    """Calculate cosine similarity between each sentence pair using Doc2Vec embeddings
    
    Args:
        df (pandas dataframe): dataframe as provided by the nlp_utils
        stop_words (bool): whether to use stop words (True) or remove them (False)
    
    Returns:
        list: predicted values for sentence similarity of test set examples
    """
    predictions = []
    if rm_stopwords:
        tokenized_sentences = zip(
            df["sentence1_tokens_stop"], df["sentence2_tokens_stop"]
        )
    else:
        tokenized_sentences = zip(df["sentence1_tokens"], df["sentence2_tokens"])

    labeled_questions = []
    sentences = list(tokenized_sentences)
    # doc2vec requires data as Tagged Documents with the tokenized sentence and the sentence id
    for i in df.index:
        labeled_questions.append(
            TaggedDocument(sentences[i][0], df[df.index == i].qid1)
        )
        labeled_questions.append(
            TaggedDocument(sentences[i][1], df[df.index == i].qid2)
        )

    # instantiate Doc2Vec model
    model = Doc2Vec(
        labeled_questions, dm=1, min_count=1, window=5, vector_size=500, epochs=30
    )

    # Train our model for 20 epochs
    for epoch in range(20):
        model.train(
            labeled_questions, epochs=model.epochs, total_examples=model.corpus_count
        )

    # Get similarity between all sentence pairs in our dataset
    for (sentence1, sentence2) in sentences:
        if len(sentence1) == 0 or len(sentence2) == 0:
            predictions.append(0)
            continue
        score = model.wv.n_similarity(sentence1, sentence2)
        predictions.append(score)
    return predictions

In [34]:
baselines["Doc2vec Cosine"] = doc2vec_cosine(sts_test_stop, rm_stopwords=True)
baselines["Doc2vec Cosine with Stop Words"] = doc2vec_cosine(sts_test_stop, rm_stopwords=False)

# Comparison of Baseline Models

Our evaluation metric is Pearson correlation ($\rho$) which is a measure of the linear correlation between two variables. The formula for calculating Pearson correlation is as follows:  

$$\rho_{X,Y} = \frac{E[(X-\mu_X)(Y-\mu_Y)]}{\sigma_X \sigma_Y}$$

This metric takes a value in [-1,1] where -1 represents a perfect negative correlation, 1 represents a perfect positive correlation, and 0 represents no correlation. We utilize the Pearson correlation metric as this is the metric that [SentEval](http://nlpprogress.com/english/semantic_textual_similarity.html), a widely-used evaluation toolkit for evaluation sentence representations, uses for the STS Benchmark dataset.

In [28]:
def pearson_correlation(df, prediction):
    """Calculate the Pearson correlation between two vectors
    
    Args:
        df (pandas dataframe): dataframe of sentences and their similarity scores
        prediction (list): predicted similarity scores for each value in test set
        
    Returns:
        float: pearson correlation value between the actual and predicted score lists
    """
    pearson_correlation = scipy.stats.pearsonr(prediction, list(df["score"]))[0]
    return pearson_correlation

In [109]:
# Get metrics on predictions from all models
for model in baselines:
    print(model, pearson_correlation(sts_test_stop, baselines[model]))

TF-IDF Cosine 0.7034695168223283
TF-IDF Cosine with Stop Words 0.6683811410442564


We investigate our 8 models with and without stop words (16 different results today). The results show that TF-IDF bag-of-words document embeddings (without stop words) combined with the cosine similarity performs the best, with a Pearson correlation of 0.7034. 